# Importing the Libraries

In [ ]:
import pandas as pd
import numpy as np

#Libabries for GeoPlots
import geoviews as gv
import geoviews.feature as gf
import geopandas as gpd
from cartopy import crs as ccrs

#Libraries for ScatterPlots
import holoviews as hv
from holoviews import opts, dim

#Library to Save the Plots
import panel as pn

#Libraries for Table
from bokeh.io import show, save, output_file
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn

hv.extension('bokeh')
gv.extension('bokeh')

## Importing and Rearranging the Data

In [ ]:
#Import and Rearrange DataFrames
##Happiness Database
happiness_data_all = pd.read_csv('World_Happiness_Report_2019.csv')
happiness_data = happiness_data_all.drop(columns={'Country','GDP per capita','Social support','Healthy life expectancy',
                                              'Freedom to make life choices','Generosity','Perceptions of corruption'})
##HDI Database - also includes GNI_PPP$
gni_data_all = pd.read_csv('GNI_2018.csv')
gni_data = gni_data_all.drop(columns={'Country'})

##GeoPandas Database
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={'iso_a3':'ISO3'})
###Correcting GeoPandas Database
world.loc[world['name']=='Norway','ISO3'] = 'NOR'
world.loc[world['name']=='France','ISO3'] = 'FRA'
world.loc[world['name']=='Kosovo','ISO3'] = 'RKS'
world.loc[world['name']=='Kosovo','ISO3'] = 'RKS'

df_init = world.merge(happiness_data, on= 'ISO3', how='left')
df = df_init.merge(gni_data, on= 'ISO3', how='left')
df = df.rename(columns={'Score':'Happiness_Score','name':'Country',
                        'Gross national income (GNI) per capita (PPP k$)':'GNI_per_capita_PPP'})

#Reordering columns
order = [0,1,2,3,4,6,7,8,9,5] # setting column's order
df = df[[df.columns[i] for i in order]]

## Creating the GeoMap Plots

In [ ]:
#Instatiating the Figures using Geoviews
##Happiness Map
df = df.rename(columns={'Rank_Happiness':'Rank'}) #Need to rename columns to have correct Rank
happiness_map = gv.Polygons(df, vdims=['Happiness_Score','Rank','Country'],label= 'Happiness by Country, 2019'
           ).opts(projection=ccrs.Robinson(), width=800, height=400,tools=['hover'], colorbar=True, cmap='RdYlGn')

##GNI Map
df = df.rename(columns={'Rank':'Rank_Happiness','Rank_GNI_PPP':'Rank'}) #Need to rename columns to have correct Rank
gni_map = gv.Polygons(df, vdims=['GNI_per_capita_PPP','Rank','Country'],label= 'GNI per capita PPP (in k$) by Country, 2018'
           ).opts(projection=ccrs.Robinson(), width=800, height=400,tools=['hover'], colorbar=True, cmap='Spectral')

In [ ]:
#Saving the Figures to HTML format
##Happiness
p_happiness = pn.panel(happiness_map)
p_happiness.save('Happiness_Map.html')

##GNI
p_happiness = pn.panel(gni_map)
p_happiness.save('GNI_Map.html')

## Creating Scatter Plot

In [ ]:
#Rearrange the DataFrame
df_scatter = pd.DataFrame(df)
df_scatter = df_scatter.drop(columns={'pop_est','ISO3','gdp_md_est','Rank_Happiness','Rank','geometry'})
df_scatter = df_scatter.dropna()

In [ ]:
#Scatter Plot for Happiness vs GNI
Happiness_GNI_scatter = hv.Scatter(df_scatter, kdims = ['Happiness_Score', 'GNI_per_capita_PPP'])

##Colors
explicit_mapping = {'Europe': 'lime', 'Asia': 'yellow', 'Oceania': 'red', 'North America': 'navy','South America': 'aqua',
                   'Africa':'saddlebrown'}

##Defining the Charts's Area

x_range = (df_scatter['Happiness_Score'].min()-0.2,df_scatter['Happiness_Score'].max()+0.2)
y_range = (df_scatter['GNI_per_capita_PPP'].min()-3,df_scatter['GNI_per_capita_PPP'].max()+3)
              
##Plot Joining all together
Happiness_GNI_scatter = Happiness_GNI_scatter.opts(opts.Scatter(tools=['hover'], height = 600, width=800, size = 10, 
                                                                xlim = x_range, ylim = y_range,
                                   color = 'continent', cmap=explicit_mapping, legend_position = 'top'))

# Create the Trendline
x = df_scatter['Happiness_Score']
y = df_scatter['GNI_per_capita_PPP']
par = np.polyfit(x, y, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in x]

trendline = hv.Curve((x, y_predicted)).opts(opts.Curve(color='black'))

full_scatter_GNI = (Happiness_GNI_scatter * trendline).opts(title="Happiness and Wealth by Country")

#Save ScatterPlot to html file
p = pn.panel(full_scatter_GNI)
p.save('Happiness_vs_Wealth.html', embed = True)

# Save the DataFrame to html format

In [ ]:
#DataFrame for Table
happiness_data_all = pd.read_csv('World_Happiness_Report_2019.csv')
happiness_data = happiness_data_all.drop(columns={'GDP per capita','Social support','Healthy life expectancy',
                                              'Freedom to make life choices','Generosity','Perceptions of corruption'})

df_table = gni_data.merge(happiness_data, on = 'ISO3', how='left')
df_table = df_table.drop(columns={'ISO3'})
df_table = df_table.rename(columns={'Score':'Happiness Score',
                                    'Rank_Happiness': 'Rank Happiness',
                                   'Gross national income (GNI) per capita (PPP k$)':'GNI per Capita PPP',
                                   'Rank_GNI_PPP':'Rank GNI per capita PPP'}) 
#Reordering columns
order = [2,3,4,0,1] # setting column's order
df_table = df_table[[df_table.columns[i] for i in order]]

df_table = df_table.dropna()
df_table = df_table.round({'Happiness Score':3, 'GNI per Capita PPP': 3})

In [ ]:
df_columns = df_table.columns

#Define the Source and the Columns of the DataFrame
source = ColumnDataSource(df_table)

columns = [TableColumn(field=Ci, title=Ci, width=20) for Ci in df_columns]

#Create the DataFrame
data_table = DataTable(source=source, columns=columns, width=700, height=400, selectable = True, index_position = None)

#Save the DataFrame
output_file('Data_Table.html')
save(data_table)